In [47]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.layers import Embedding

from keras.layers import Multiply

from keras.optimizers import Adam

from keras.initializers import truncated_normal , random_normal , constant

In [42]:
import os

import matplotlib as plt
import numpy as np

import pandas as pd

import gc

%matplotlib inline

In [15]:
#CIFAR10 dataset
WIDTH = 32
HEIGHT = 32
CHANNEL = 3

LATENT_DIM = 100 #latent variable z sample from normal distribution

BATCH_SIZE = 64
EPOCHS = 10

PATH = '../dataset/train/'

#生成多少个图像 长*宽
ROW = 5
COL = 5

#=========
#=========
#add new FLAG(s)
CLASS_NUM = 10 #mnist=10 CIFAR10=10 CIFAR100=100 CIFAR1000=1000

LABEL2INDEX = {'frog':0 , 'truck':1 , 'deer':2 , 'automobile':3 , 'bird':4 , 'horse':5 , 'ship':6 , 'cat':7 , 'dog':8 , 'airplane':9}
INDEX2LABEL = {value:key for key , value in LABEL2INDEX.items()}

In [16]:
INDEX2LABEL

{0: 'frog',
 1: 'truck',
 2: 'deer',
 3: 'automobile',
 4: 'bird',
 5: 'horse',
 6: 'ship',
 7: 'cat',
 8: 'dog',
 9: 'airplane'}

In [39]:

load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)


In [27]:
train_labels = pd.read_csv('../dataset/trainLabels.csv')
#每张图片对应的类别标号
train_labels = train_labels['label'].map(LABEL2INDEX).get_values()

In [5]:
'''
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)
y_train = np.expand_dims(y_train , 1)
'''

In [55]:
'''
def load_mnist():
    random_index = np.random.randint(0, X_train.shape[0], BATCH_SIZE)
    return X_train[random_index] , y_train[random_index]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    label = np.random.randint(low=0 , high=CLASS_NUM , size=(ROW*COL , 1)) #CLASS_NUM 是否需要加1
    
    generated_image = generator_i.predict([noise , label])
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    fig.subplots_adjust(hspace=0.9 , wspace=0.9) #将每一个图像在高度 宽度 上进行缩放或者放大
    #fig.tight_layout()
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            axes[i][j].set_title('%d' % (label[i*ROW+j]))
            count += 1
    
    
    fig.savefig('mnist_acgan/No.%d.png' % (epoch))
    plt.pyplot.close()
'''

In [67]:

def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    labels = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
        labels.append(train_labels[(load_index + i) % IMAGES_COUNT])
    
    load_index += batch_size
    
    return np.array(images)/127.5-1 , np.array(labels)

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    labels = np.random.randint(low=0 , high=CLASS_NUM , size=(ROW*COL , 1))
    
    generated_image = generator_i.predict([noise , labels])
    
    generated_image = (generated_image+1)*127.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    fig.subplots_adjust(hspace=0.9 , wspace=0.9)
    
    count = 0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            axes[i][j].set_title(INDEX2LABEL[labels[i*ROW+j][0]])
            
            count += 1
            
    fig.savefig('CIFAR10_acgan/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')


In [44]:
def conv2d(output_size):
    return Conv2D(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0))

def dense(output_size):
    return Dense(output_size , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def deconv2d(output_size):
    return Conv2DTranspose(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def batch_norm():
    return BatchNormalization(momentum=0.9 , epsilon=1e-5)


In [54]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    #CIFAR10 图像使用 32*32*3
    model.add(Dense(2 * 2 * 64*8, activation="relu", input_shape=(LATENT_DIM,)))
    model.add(Reshape((2, 2, 64*8)))
    
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #4
    model.add(deconv2d(64*4))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #8
    model.add(deconv2d(64*2))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #16
    model.add(deconv2d(64*1))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #32
    model.add(deconv2d(3))
    model.add(Activation('tanh'))
    
    #model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    label = Input(shape=(1,) , dtype='int32')
    
    _ = Embedding(input_dim=CLASS_NUM , output_dim=LATENT_DIM)(label)
    embedding_label = Flatten()(_)
    
    noise_embedding_label = Multiply()([noise , embedding_label]) #(None , LATENT_DIM)
    
    image = model(noise_embedding_label)
    
    return Model([noise , label] , image , name='generator_Model')

In [80]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Conv2D(filters=64 , kernel_size=(5,5) , strides=(2,2) , padding='same' , input_shape=(WIDTH , HEIGHT , CHANNEL) , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0) , name='conv1'))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(conv2d(64*2))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(conv2d(64*4))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))

    model.add(conv2d(64*8))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    
    #model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    flatten_feature = model(image)
    
    #FC层 多加了一层
    #_ = Dense(128)(flatten_feature)
    #_ = LeakyReLU(alpha=0.2)(_)
    
    validity = Dense(1 , activation='sigmoid')(flatten_feature)
    
    label_hat = Dense(CLASS_NUM , activation='softmax')(flatten_feature) #CLASS_NUM位置是否需要加1 双输出 双损失函数 为每一个输出指定损失权重
    
    return Model(image , [validity , label_hat] , name='discriminator_Model')

In [81]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    label = Input(shape=(1,) , dtype='int32')
    
    image = generator_i([z , label])
    
    #print(image.shape)
    
    discriminator_i.trainable = False
    validity , label_hat = discriminator_i(image)
    
    return Model([z , label] , [validity , label_hat] , name='combined_model')

In [82]:
adam = Adam(lr = 0.0002 , beta_1=0.5)

In [83]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=adam , loss=['binary_crossentropy' , 'sparse_categorical_crossentropy'] , metrics=['accuracy'])

generator_i = generator()

combined_model_i = combined_model(generator_i , discriminator_i)


combined_model_i.compile(optimizer=adam , loss=['binary_crossentropy' , 'sparse_categorical_crossentropy'])

In [84]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(10000):
    for j in range(int(IMAGES_COUNT/BATCH_SIZE)):
        
        noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
        corresponding_fake_label = np.random.randint(low=0 , high=CLASS_NUM , size=(BATCH_SIZE , 1)) #label的取值范围 可能会发生变化

        real_image , corresponding_real_label = load_image()

        #训练判别器
        fake_image = generator_i.predict([noise , corresponding_fake_label])

        real_loss = discriminator_i.train_on_batch(real_image , [real_labels , corresponding_real_label])
        fake_loss = discriminator_i.train_on_batch(fake_image , [fake_labels , corresponding_fake_label])

        loss = np.add(real_loss , fake_loss)/2

        #训练生成器
        noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
        corresponding_fake_label2 = np.random.randint(low=0 , high=CLASS_NUM , size=(BATCH_SIZE , 1))

            #下面的损失是一个list 有两个损失 一个是validity一个是与label的softmax
        generator_loss = combined_model_i.train_on_batch([noise2 , corresponding_fake_label2] , [real_labels , corresponding_fake_label2])

        print('epoch:%d batch:%d loss:%f accu:%f gene_loss:[validity:%f softmax:%f]' % (i , j , loss[0] , loss[1] , generator_loss[0] , generator_loss[1]))

        #if j % 1000 == 0:
    write_image(i)
    
write_image(999)


C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 batch:0 loss:3.760308 accu:0.988528 gene_loss:[validity:5.268500 softmax:2.495670]
epoch:0 batch:1 loss:3.394735 accu:0.212073 gene_loss:[validity:7.514019 softmax:5.140787]
epoch:0 batch:2 loss:4.086455 accu:0.141819 gene_loss:[validity:7.463078 softmax:5.072567]
epoch:0 batch:3 loss:3.841987 accu:0.173248 gene_loss:[validity:8.531496 softmax:5.757126]
epoch:0 batch:4 loss:3.785560 accu:0.049021 gene_loss:[validity:8.355479 softmax:5.620314]
epoch:0 batch:5 loss:3.271908 accu:0.094353 gene_loss:[validity:8.658384 softmax:5.369433]
epoch:0 batch:6 loss:3.094976 accu:0.057955 gene_loss:[validity:7.097963 softmax:4.550370]
epoch:0 batch:7 loss:2.802643 accu:0.112917 gene_loss:[validity:8.200959 softmax:5.704834]
epoch:0 batch:8 loss:2.592715 accu:0.038125 gene_loss:[validity:7.456964 softmax:4.952616]
epoch:0 batch:9 loss:2.686116 accu:0.098243 gene_loss:[validity:7.503994 softmax:4.979907]
epoch:0 batch:10 loss:2.553970 accu:0.049594 gene_loss:[validity:6.632004 softmax:4.312901

epoch:0 batch:91 loss:3.429505 accu:1.277523 gene_loss:[validity:3.000068 softmax:1.095196]
epoch:0 batch:92 loss:3.157656 accu:0.952507 gene_loss:[validity:3.207081 softmax:1.214532]
epoch:0 batch:93 loss:2.899544 accu:0.754067 gene_loss:[validity:3.207517 softmax:1.361648]
epoch:0 batch:94 loss:2.744381 accu:0.699134 gene_loss:[validity:3.102299 softmax:1.262866]
epoch:0 batch:95 loss:2.795307 accu:0.679804 gene_loss:[validity:3.382185 softmax:1.498739]
epoch:0 batch:96 loss:2.842963 accu:0.739782 gene_loss:[validity:3.498563 softmax:1.559895]
epoch:0 batch:97 loss:3.184525 accu:1.004970 gene_loss:[validity:3.115542 softmax:1.094861]
epoch:0 batch:98 loss:2.864079 accu:0.752414 gene_loss:[validity:3.096129 softmax:1.226890]
epoch:0 batch:99 loss:2.885808 accu:0.736986 gene_loss:[validity:3.198530 softmax:1.384292]
epoch:0 batch:100 loss:2.927258 accu:0.806950 gene_loss:[validity:3.264295 softmax:1.416292]
epoch:0 batch:101 loss:3.053257 accu:0.871806 gene_loss:[validity:2.931567 soft

epoch:0 batch:180 loss:2.828806 accu:0.773933 gene_loss:[validity:2.709238 softmax:1.046173]
epoch:0 batch:181 loss:2.759013 accu:0.691577 gene_loss:[validity:2.962741 softmax:1.088474]
epoch:0 batch:182 loss:2.763084 accu:0.754265 gene_loss:[validity:2.751386 softmax:0.984071]
epoch:0 batch:183 loss:2.962924 accu:0.796409 gene_loss:[validity:3.153475 softmax:1.178791]
epoch:0 batch:184 loss:3.066063 accu:0.936289 gene_loss:[validity:2.847092 softmax:0.978325]
epoch:0 batch:185 loss:2.975841 accu:0.855474 gene_loss:[validity:2.648145 softmax:0.855695]
epoch:0 batch:186 loss:2.739813 accu:0.686427 gene_loss:[validity:2.955101 softmax:1.087271]
epoch:0 batch:187 loss:2.798620 accu:0.794519 gene_loss:[validity:2.705492 softmax:0.902615]
epoch:0 batch:188 loss:2.781519 accu:0.767219 gene_loss:[validity:2.713366 softmax:1.115164]
epoch:0 batch:189 loss:2.780500 accu:0.707652 gene_loss:[validity:2.998809 softmax:1.136492]
epoch:0 batch:190 loss:3.024697 accu:0.784086 gene_loss:[validity:2.63

epoch:0 batch:270 loss:2.739297 accu:0.743438 gene_loss:[validity:2.630778 softmax:0.979330]
epoch:0 batch:271 loss:2.827179 accu:0.738071 gene_loss:[validity:2.527203 softmax:0.916237]
epoch:0 batch:272 loss:2.806384 accu:0.768208 gene_loss:[validity:2.456402 softmax:0.926949]
epoch:0 batch:273 loss:2.806122 accu:0.730621 gene_loss:[validity:2.635045 softmax:0.927219]
epoch:0 batch:274 loss:2.774925 accu:0.724093 gene_loss:[validity:2.699153 softmax:0.979865]
epoch:0 batch:275 loss:2.884797 accu:0.759814 gene_loss:[validity:2.957687 softmax:0.987936]
epoch:0 batch:276 loss:2.882307 accu:0.840638 gene_loss:[validity:2.648490 softmax:0.862575]
epoch:0 batch:277 loss:2.729480 accu:0.761320 gene_loss:[validity:2.598755 softmax:0.885463]
epoch:0 batch:278 loss:2.730468 accu:0.722012 gene_loss:[validity:2.659869 softmax:1.000902]
epoch:0 batch:279 loss:2.890515 accu:0.773121 gene_loss:[validity:3.052849 softmax:1.102754]
epoch:0 batch:280 loss:3.060711 accu:0.902759 gene_loss:[validity:2.63

...

epoch:14 batch:502 loss:1.984796 accu:0.718060 gene_loss:[validity:0.760384 softmax:0.740867]
epoch:14 batch:503 loss:1.898388 accu:0.706008 gene_loss:[validity:0.765243 softmax:0.739010]
epoch:14 batch:504 loss:1.945138 accu:0.713436 gene_loss:[validity:0.800650 softmax:0.746177]
epoch:14 batch:505 loss:1.950857 accu:0.705160 gene_loss:[validity:0.805895 softmax:0.788840]
epoch:14 batch:506 loss:1.961969 accu:0.705458 gene_loss:[validity:0.801306 softmax:0.779381]
epoch:14 batch:507 loss:1.969986 accu:0.695103 gene_loss:[validity:0.805591 softmax:0.778248]
epoch:14 batch:508 loss:1.919019 accu:0.699074 gene_loss:[validity:0.803125 softmax:0.783648]
epoch:14 batch:509 loss:1.941438 accu:0.695974 gene_loss:[validity:0.800777 softmax:0.779048]
epoch:14 batch:510 loss:1.918071 accu:0.699789 gene_loss:[validity:0.822526 softmax:0.771718]
epoch:14 batch:511 loss:2.010193 accu:0.707134 gene_loss:[validity:0.820935 softmax:0.783577]
epoch:14 batch:512 loss:1.923380 accu:0.704604 gene_loss:[va

epoch:14 batch:590 loss:1.945509 accu:0.710732 gene_loss:[validity:0.820036 softmax:0.791721]
epoch:14 batch:591 loss:1.975384 accu:0.720758 gene_loss:[validity:0.810799 softmax:0.768691]
epoch:14 batch:592 loss:1.927546 accu:0.709545 gene_loss:[validity:0.790527 softmax:0.770006]
epoch:14 batch:593 loss:1.823048 accu:0.691864 gene_loss:[validity:0.813650 softmax:0.800448]
epoch:14 batch:594 loss:1.912871 accu:0.686407 gene_loss:[validity:0.822631 softmax:0.803100]
epoch:14 batch:595 loss:1.956572 accu:0.706392 gene_loss:[validity:0.809943 softmax:0.783526]
epoch:14 batch:596 loss:1.903846 accu:0.703069 gene_loss:[validity:0.819738 softmax:0.792446]
epoch:14 batch:597 loss:1.903953 accu:0.709838 gene_loss:[validity:0.812062 softmax:0.786311]
epoch:14 batch:598 loss:1.952184 accu:0.710171 gene_loss:[validity:0.792965 softmax:0.763498]
epoch:14 batch:599 loss:1.913675 accu:0.697756 gene_loss:[validity:0.841713 softmax:0.805019]
epoch:14 batch:600 loss:2.011991 accu:0.717544 gene_loss:[va

epoch:14 batch:678 loss:1.912603 accu:0.710208 gene_loss:[validity:0.800905 softmax:0.781773]
epoch:14 batch:679 loss:1.960810 accu:0.704977 gene_loss:[validity:0.852236 softmax:0.826709]
epoch:14 batch:680 loss:1.927499 accu:0.728237 gene_loss:[validity:0.801201 softmax:0.772428]
epoch:14 batch:681 loss:1.912694 accu:0.702531 gene_loss:[validity:0.837850 softmax:0.812315]
epoch:14 batch:682 loss:1.918194 accu:0.721257 gene_loss:[validity:0.809112 softmax:0.785808]
epoch:14 batch:683 loss:1.957632 accu:0.708622 gene_loss:[validity:0.800888 softmax:0.778939]
epoch:14 batch:684 loss:1.984499 accu:0.714431 gene_loss:[validity:0.779853 softmax:0.753579]
epoch:14 batch:685 loss:1.876105 accu:0.691652 gene_loss:[validity:0.789577 softmax:0.774300]
epoch:14 batch:686 loss:1.954744 accu:0.687437 gene_loss:[validity:0.867988 softmax:0.796141]
epoch:14 batch:687 loss:1.980135 accu:0.735117 gene_loss:[validity:0.918375 softmax:0.819693]
epoch:14 batch:688 loss:1.993658 accu:0.772132 gene_loss:[va

epoch:14 batch:766 loss:1.945279 accu:0.697630 gene_loss:[validity:0.839873 softmax:0.795502]
epoch:14 batch:767 loss:1.953078 accu:0.723145 gene_loss:[validity:0.787908 softmax:0.763701]
epoch:14 batch:768 loss:1.938409 accu:0.698702 gene_loss:[validity:0.795497 softmax:0.775326]
epoch:14 batch:769 loss:1.891819 accu:0.695947 gene_loss:[validity:0.785470 softmax:0.765769]
epoch:14 batch:770 loss:1.911544 accu:0.691045 gene_loss:[validity:0.803522 softmax:0.781108]
epoch:14 batch:771 loss:1.960147 accu:0.701121 gene_loss:[validity:0.815441 softmax:0.779242]
epoch:14 batch:772 loss:1.988571 accu:0.708081 gene_loss:[validity:0.806858 softmax:0.759459]
epoch:14 batch:773 loss:1.926895 accu:0.693799 gene_loss:[validity:0.808700 softmax:0.777260]
epoch:14 batch:774 loss:2.022930 accu:0.700028 gene_loss:[validity:0.812480 softmax:0.768194]
epoch:14 batch:775 loss:1.915771 accu:0.694256 gene_loss:[validity:0.806080 softmax:0.784839]
epoch:14 batch:776 loss:1.886966 accu:0.684442 gene_loss:[va

KeyboardInterrupt: 

In [70]:
real_labels.shape

(64, 1)

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [39]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         29029120  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 14320641  
Total params: 43,349,761
Trainable params: 29,025,536
Non-trainable params: 14,324,225
_________________________________________________________________


In [ ]:
gc.collect()

In [41]:
32*400

12800

In [ ]:
gc.collect()